In [14]:
import sys
import os
from git_root import git_root

import numpy as np
import pandas as pd
import tensorflow as tf

sys.path.append(git_root("utils"))
from utils import load_params, load_config

sys.path.append(git_root("training"))
from fetch_data import fetch_data_local, fetch_format_stacked
from fetch_data import to_numpy_arrays, prepare_tf_dataset

import json

print(tf.__version__)

2.0.0


# Generating the predictions for the test sets

In [15]:
params = load_params()

angles = params["GLCM"]["mel_map"]["angles_in_deg"]

place_holder = {
    "spectrogram": {0: []},
    "mel_map": {
        angle: [] for angle in angles
    }
}

In [16]:
from tensorflow.keras.models import load_model
sys.path.append(git_root("models"))
from affine_scalar_layer import AffineScalar


mapping_for_saved_models = {'spectrogram': 'spec',
                           'mel_map': 'mel'}

for map_type in place_holder:
    for angle in place_holder[map_type]:
        #Fetch the data
        print(f"Map: {map_type}, angle: {angle}")
        df = fetch_data_local(
            map_type=map_type, train=False, angle=angle
        )
        #Fetch the trained model
        model_path = git_root('models','saved_models',
                                  'glcm_{}_angle{}.h5'.format(mapping_for_saved_models[map_type],angle))
        net = load_model(model_path,  custom_objects={'AffineScalar':AffineScalar()})
        #For some reason, I need to cast my tensor to float32 for predict (and not for fit ?!)
        dataset = tf.data.Dataset.from_tensors(tf.dtypes.cast(to_numpy_arrays(df)[0], tf.float32))
        #Generate predictions
        y = net.predict(dataset)
        preds = pd.DataFrame(y)
        preds.columns = ["class{}".format(i) for i in range(10)]
        
        #Concatenate the prediction to our dataset
        df = df.merge(preds, left_index=True, right_index=True)
        df = df.drop(['maps'], axis=1)
        
        save_path=git_root("evaluation","evaluation_results","results_{}_{}.csv".format(map_type,angle))
        print("Saving to: {}".format(save_path))
        df.to_csv(save_path)

Map: spectrogram, angle: 0
Fetching: data_spectrogram_angle_0_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_spectrogram_0.csv
Map: mel_map, angle: 0
Fetching: data_mel_map_angle_0_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_mel_map_0.csv
Map: mel_map, angle: 45
Fetching: data_mel_map_angle_45_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_mel_map_45.csv
Map: mel_map, angle: 90
Fetching: data_mel_map_angle_90_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_mel_map_90.csv
Map: mel_map, angle: 135
Fetching: data_mel_map_angle_135_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_mel_map_135.csv


In [17]:
fetch_data_local(
            map_type='mel_map', train=False, angle=angle
        )

Fetching: data_mel_map_angle_135_test.json


,filename,maps,split_id,genre
0,blues.00018.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",0,blues
1,blues.00018.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",1,blues
2,blues.00018.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",2,blues
3,blues.00018.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",3,blues
4,blues.00018.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",4,blues
...,...,...,...,...
1395,rock.00096.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",9,rock
1396,rock.00096.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",10,rock
1397,rock.00096.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",11,rock
1398,rock.00096.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",12,rock


In [18]:
temp = []

for i in range(len(angles)):
    temp.append(fetch_data_local(
            map_type='mel_map', train=False, angle=angles[i]
        )['maps'])

stacked_mel_maps = (
    [
        np.stack([
            np.squeeze(temp[j][i]) for j in range(len(angles))
        ], -1) for i in range(1400)
    ]
)

Fetching: data_mel_map_angle_0_test.json
Fetching: data_mel_map_angle_45_test.json
Fetching: data_mel_map_angle_90_test.json
Fetching: data_mel_map_angle_135_test.json


In [19]:
#For stacked model

#Fetch the trained model
model_path = git_root('models','saved_models',
                                  'glcm_{}_stacked.h5'.format(mapping_for_saved_models[map_type],angle))
net = load_model(model_path,  custom_objects={'AffineScalar':AffineScalar()})
#For some reason, I need to cast my tensor to float32 for predict (and not for fit ?!)
#dataset = tf.data.Dataset.from_tensors(tf.dtypes.cast(fetch_format_stacked(train=False), tf.float32))
dataset = tf.data.Dataset.from_tensor_slices(tf.dtypes.cast(stacked_mel_maps, tf.float32)).batch(32)
#Generate predictions
y = net.predict(dataset)
preds = pd.DataFrame(y)
preds.columns = ["class{}".format(i) for i in range(10)]
        
#Concatenate the prediction to our dataset
df = fetch_data_local(map_type='mel_map', train=False, angle='0').loc[:,['filename','split_id','genre']].merge(preds,left_index=True, right_index=True)
        
save_path=git_root("evaluation","evaluation_results","results_{}_{}.csv".format('mel_map','stacked'))
print("Saving to: {}".format(save_path))
df.to_csv(save_path)

Fetching: data_mel_map_angle_0_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_mel_map_stacked.csv


# Results

In [20]:
config = load_config()

label_names = config["genres"]
label_to_idx = dict((name, index) for index, name in enumerate(label_names))
idx_to_label = {v: k for k, v in label_to_idx.items()}

class_columns = ["class{}".format(i) for i in range(10)]

**Per Piece**

In [21]:
from sklearn.metrics import classification_report

piece_level_acc = {"spectrogram": {0: {}},
    "mel_map": {
        angle: {} for angle in angles
    }}

piece_level_acc['mel_map']['stacked'] = {}

for map_type in piece_level_acc:
    for angle in piece_level_acc[map_type]:
        filepath=git_root("evaluation","evaluation_results","results_{}_{}.csv".format(map_type,angle))
        df = pd.read_csv(filepath, index_col=0)
        
        #Generate the label by taking highest prob
        df['pred'] = np.argmax(df[class_columns].values, axis=1)
        
        df['label'] = df['genre'].apply(lambda x: label_to_idx[x])
        
        report = classification_report(df['label'], df['pred'], output_dict=True)
        piece_level_acc[map_type][angle]['accuracy'] = report['accuracy']
        
        piece_level_acc[map_type][angle]['class_precision'] = {}
        
        for i in range(10):
            piece_level_acc[map_type][angle]['class_precision'][idx_to_label[i]] = report[str(i)]['precision']

In [22]:
#Dump the results
with open(git_root('evaluation','evaluation_results','glcm_piece_level_results.json'), 'w') as fp:
    json.dump(piece_level_acc, fp, indent=4)

**Per track**

In [23]:
#Generate a table that contains mapping (track: genre)
track_mapping = df[['filename','genre']].drop_duplicates().reset_index(drop=True)

In [24]:
track_level_acc = {"spectrogram": {0: {}},
    "mel_map": {
        angle: {} for angle in angles
    }}

track_level_acc['mel_map']['stacked'] = {}

for map_type in track_level_acc:
    for angle in track_level_acc[map_type]:
        filepath=git_root("evaluation","evaluation_results","results_{}_{}.csv".format(map_type,angle))
        df = pd.read_csv(filepath, index_col=0)
        #Group by track to get track_level probs
        #Note that we SUM the probs (rather than AVG them)
        df_per_track = df.groupby('filename').sum()
        #Label is the highest prob
        df_per_track['pred'] = np.argmax(df_per_track[class_columns].values, axis=1)
        #Merge back to get the track_names
        df_result = df_per_track.merge(track_mapping, on='filename')
        df_result['label'] = df_result['genre'].apply(lambda x: label_to_idx[x])
        
        report = classification_report(df_result['label'], df_result['pred'], output_dict=True)
        track_level_acc[map_type][angle]['accuracy'] = report['accuracy']
        
        track_level_acc[map_type][angle]['class_precision'] = {}
        
        for i in range(10):
            track_level_acc[map_type][angle]['class_precision'][idx_to_label[i]] = report[str(i)]['precision']

In [25]:
#Dump the results
with open(git_root('evaluation','evaluation_results','glcm_track_level_results.json'), 'w') as fp:
    json.dump(track_level_acc, fp, indent=4)

In [26]:
track_level_acc

{'spectrogram': {0: {'accuracy': 0.5,
   'class_precision': {'blues': 0.5,
    'classical': 0.9,
    'country': 1.0,
    'disco': 0.42857142857142855,
    'hiphop': 0.5,
    'jazz': 0.4666666666666667,
    'metal': 0.8181818181818182,
    'pop': 0.34782608695652173,
    'reggae': 0.4666666666666667,
    'rock': 0.09090909090909091}}},
 'mel_map': {0: {'accuracy': 0.52,
   'class_precision': {'blues': 0.8333333333333334,
    'classical': 0.8181818181818182,
    'country': 0.5,
    'disco': 0.5,
    'hiphop': 0.3333333333333333,
    'jazz': 0.5454545454545454,
    'metal': 0.7692307692307693,
    'pop': 0.3076923076923077,
    'reggae': 0.42857142857142855,
    'rock': 0.25}},
  45: {'accuracy': 0.42,
   'class_precision': {'blues': 0.5,
    'classical': 0.6153846153846154,
    'country': 0.2,
    'disco': 0.2,
    'hiphop': 0.2,
    'jazz': 0.5,
    'metal': 0.7692307692307693,
    'pop': 0.36,
    'reggae': 0.4166666666666667,
    'rock': 0.1}},
  90: {'accuracy': 0.41,
   'class_preci